# Simple Solution SymPy Code Generation

In [ ]:
import pandas as pd
import numpy as np
import sympy as sym
sym.init_printing()
from thermoengine import coder
from thermoengine import core


## Simple Solution Properties - Structure of the Equations
There are three terms:
- Terms describing standard state contributions
- Terms describing the configurational entropy of solution
- Terms describing the excess enthalpy of solution  

Assumptions:
- There are $c$ components in the system
- There are as many endmember species, $s$, as there are components
- The configurational entropy is described as a simple $x_i log(x_i)$ sum
- The excess enthalpy is described using an asymmetric regular solution $\left[ {{W_{ij}} + \Delta {W_{ij}}\left( {{X_i} - {X_j}} \right)} \right]{X_i}{X_j}$, where the $W_{ij}$ and $\Delta{W_{ij}}$ are allowed to be first order functions of both $T$ and $P$
- Ternary interaction terms are permittted, i.e. $W_{ijk}$

## Number of solution components
This notebook illustrates a three component solution

In [ ]:
c = 3

## Create a simple solution model
... with the specified number of endmember thermodynamic components

In [ ]:
model = coder.SimpleSolnModel(nc=c)

## Retrieve primary compositional variables
- $n$ is a vector of mole numbers of each component  
- $n_T$ is the total number of moles in the solution
### and construct a derived mole fraction variable
- $X$ is a vector of mole fractions of components in the system

In [ ]:
n = model.n
nT = model.nT
X = n/nT
n, nT, X

## Retrieve the temperature, pressure, and standard state chemical potentials
- $T$ is temperature in $K$
- $P$ is pressure in $bars$
- $\mu$ in Joules

In [ ]:
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
mu = model.mu
T,P,mu

## Define the standard state contribution to solution properties

In [ ]:
G_ss = (n.transpose()*mu)[0]
G_ss

## Define configurational entropy and configurational Gibbs free energy

In [ ]:
S_config,R = sym.symbols('S_config R')
S_config = 0
for i in range(0,c):
    S_config += X[i]*sym.log(X[i])
S_config *= (-R*nT)*3
S_config

In [ ]:
G_config = sym.simplify(-T*S_config)
G_config

## Parameterize the excess free energy
- Symmetric terms: $W_{ij} = Wh_{ij} - T Ws_{ij} + P Wv_{ij}$, where $Wh_{ij}$ is the excess enthalpy along the $i$-$j$ binary, $Ws_{ij}$ is the excess entropy, and $Wv_{ij}$ is the excess volume
- Asymetric terms: $\Delta W_{ij} = \Delta Wh_{ij} - T \Delta Ws_{ij} + P \Delta Wv_{ij}$
- Convention: $\left[ {{W_{ij}} + \Delta {W_{ij}}\left( {{X_i} - {X_j}} \right)} \right]{X_i}{X_j} = \left( {{W_{ij}} + \Delta {W_{ij}}} \right)X_i^2{X_j} + \left( {{W_{ij}} - \Delta {W_{ij}}} \right){X_i}X_j^2 = {W_{iij}}X_i^2{X_j} + {W_{ijj}}{X_i}X_j^2$  
- Strictly ternary terms: $W_{ijk}X_iX_jX_k$, where $i {\ne} j {\ne} k$

In [ ]:
module = 'asymm_regular'
params = []
units = []
symparam = ()
G_excess = sym.symbols('G_excess')
G_excess = 0
for i in range(1,c):
    for j in range(i+1,c+1):
        param = 'Wh' + str(i) + str(j); params.append(param); units.append('J/m')
        h_term = sym.symbols(param); symparam += (h_term,)
        param = 'Ws' + str(i) + str(j); params.append(param); units.append('J/K-m')
        s_term = sym.symbols(param); symparam += (s_term,)
        param = 'Wv' + str(i) + str(j); params.append(param); units.append('J/bar-m')
        v_term = sym.symbols(param); symparam += (v_term,)
        param = 'dWh' + str(i) + str(j); params.append(param); units.append('J/m')
        dh_term = sym.symbols(param); symparam += (dh_term,)
        param = 'dWs' + str(i) + str(j); params.append(param); units.append('J/K-m')
        ds_term = sym.symbols(param); symparam += (ds_term,)
        param = 'dWv' + str(i) + str(j); params.append(param); units.append('J/bar-m')
        dv_term = sym.symbols(param); symparam += (dv_term,)
        w_term = h_term - T*s_term + P*v_term
        dw_term = dh_term - T*ds_term + P*dv_term
        G_excess += (w_term + dw_term*(n[i-1]-n[j-1])/nT)*n[i-1]*n[j-1]
G_excess /= nT
for i in range(1,c-1):
    for j in range(i+1,c):
        for k in range(j+1,c+1):
            param = 'Wh' + str(i) + str(j) + str(k); params.append(param); units.append('J/m')
            h_term = sym.symbols(param); symparam += (h_term,)
            param = 'Ws' + str(i) + str(j) + str(k); params.append(param); units.append('J/K-m')
            s_term = sym.symbols(param); symparam += (s_term,)
            param = 'Wv' + str(i) + str(j) + str(k); params.append(param); units.append('J/bar-m')
            v_term = sym.symbols(param); symparam += (v_term,)
            G_excess += (h_term - T*s_term + P*v_term)*n[i-1]*n[j-1]*n[k-1]/nT/nT
G_excess

In [ ]:
print(params)
print(units)

## Define the Gibbs free energy of solution

In [ ]:
G = G_ss + G_config + G_excess
G

## Add the Gibbs free energy of solution to the model

In [ ]:
model.add_expression_to_model(G, list(zip(params, units, symparam)))

... give the model a unqiue name

In [ ]:
model.module = "Berman96_Solution"

In [ ]:
model.formula_string = '(Fe[Fe]Ca[Ca]Mg[Mg])Al2Si3O12'
model.conversion_string = ['[0]=[Fe]', '[1]=[Ca]', '[2]=[Mg]']
model.test_string = ['[0] > 0.0', '[1] > 0.0', '[2] > 0.0']

# Define Parameters of a Garnet Solution
Berman and Koziol, 1991 Component order
1. Grossular, ${\rm{Ca_3Al_2Si_3O_{12}}}$
2. Pyrope, ${\rm{Mg_3Al_2Si_3O_{12}}}$
3. Almandine, ${\rm{Fe_3Al_2Si_3O_{12}}}$

MELTS component ordering
1. Almandine
2. Grossular
3. Pyrope

Original calibration from the Berman and Koziol, 1991 paper using their notation:
```
wh112   = 21560.0;  /* joules     */
ws112   = 18.79;     /* joules/K   */
wv112   = 0.10;   /* joules/bar */
wh122   = 69200.0;  /* joules     */
ws122   = 18.79;     /* joules/K   */
wv122   = 0.1;   /* joules/bar */
wh113   = 20320.0;   /* joules     */
ws113   = 5.08;      /* joules     */
wv113   = 0.17;  /* joules     */
wh133   = 2620.0;  /* joules     */
ws133   = 5.08;  /* joules/bar */
wv133   = 0.09;  /* joules     */
wh223   = 230.0;   /* joules/bar */
ws223   = 0.0;   /* joules/bar */
wv223   = 0.01;   /* joules/bar */
wh233   = 3720.0;   /* joules/bar */
ws223   = 0.0;   /* joules/bar */
wv223   = 0.06;   /* joules/bar */
```

Note that in the notation derveloped in this paper, $\left[ {{W_{ij}} + \Delta {W_{ij}}\left( {{X_i} - {X_j}} \right)} \right]{X_i}{X_j}$, is related to the convention used in the original paper: ${{\tilde W}_{13}}{X_1}{X_3}\left( {{X_3} + \frac{{{X_2}}}{2}} \right) + {{\tilde W}_{31}}{X_1}{X_3}\left( {{X_1} + \frac{{{X_2}}}{2}} \right)$:   
- ${W_{13}} = {{\tilde W}_{13}} + {{\tilde W}_{31}}$
- $d{W_{13}} = {{\tilde W}_{31}} - {{\tilde W}_{13}}$

In [ ]:
print (params)
wh21   = 33470
ws21   = 18.79
wv21   = 0.173
wh12   = 68280
ws12   = 18.79
wv12   = 0.036
wh31   = 21951.4
ws31   = 9.43
wv31   = 0.170
wh13   = 11581.5
ws13   = 9.43
wv13   = 0.09
wh32   = 5064.5
ws32   = 4.11
wv32   = 0.1
wh23   = 6269.1
ws23   = 4.11
wv23   = 0.06
paramValues = { 'Wh12':(wh13+wh31)/2,  'Ws12':(ws31+ws13)/2, 'Wv12':(wv31+wv13)/2, \
                'Wh13':(wh23+wh32)/2,  'Ws13':(ws32+ws23)/2,  'Wv13':(wv32+wv23)/2, \
                'Wh23':(wh12+wh21)/2,  'Ws23':(ws21+ws12)/2, 'Wv23':(wv21+wv12)/2, \
                'dWh12':(wh13-wh31)/2, 'dWs12':(ws13-ws31)/2, 'dWv12':(wv13-wv31)/2, \
                'dWh13':(wh23-wh32)/2, 'dWs13':(ws23-ws32)/2, 'dWv13':(wv23-wv32)/2, \
                'dWh23':(wh21-wh12)/2, 'dWs23':(ws21-ws12)/2, 'dWv23':(wv21-wv12)/2,\
                'Wh123':73298.3, 'Ws123':32.33, 'Wv123':0.281, \
                'T_r':298.15, 'P_r':1.0}
#paramValues = { 'Wh12':wh31+wh13,  'Ws12':ws31+ws13, 'Wv12':wv31+wv13, \
                #'Wh13':wh32+wh23,  'Ws13':ws32+ws23,  'Wv13':wv32+wv23, \
                #'Wh23':wh21+wh12,  'Ws23':ws21+ws12, 'Wv23':wv21+wv12, \
                #'dWh12':wh31-wh13, 'dWs12':ws31-ws13, 'dWv12':wv31-wv13, \
                #'dWh13':wh32-wh23, 'dWs13':ws32-ws23, 'dWv13':wv32-wv23, \
                #'dWh23':wh21-wh12, 'dWs23':ws21-ws12, 'dWv23':wv21-wv12,\
                #'Wh123':0, 'Ws123':0.0, 'Wv123':0, \
                #'T_r':298.15, 'P_r':1.0}

print (paramValues)

Generate both fast computation and calibibration code for the feldspar solution

# Use code printers to construct "C" package code

In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

## Choose model type and create model
model_type is "fast" or "calib"

In [ ]:
model_type = "calib"

In [ ]:
model.create_code_module(phase="Garnet", params=paramValues, 
                         endmembers=['Almandine_Garnet', 'Grossular_Garnet', 'Pyrope_Garnet'], 
                         prefix="cy", module_type=model_type, silent=False)

## Load the module

In [ ]:

import Berman96_Solution
%cd ..

## Test and time the generated functions for Garnet (T in K, P in bars)

In [ ]:
t = 2000.00
p = 1.0
n = np.array([1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7])

## Available in both "Fast" and "Calib" code versions 
Execute the "fast" or "calibration" code metadata retrieval functions:

In [ ]:
try:
    print(Garnet_Solution.cy_Garnet_Garnet_Solution_identifier())
    print(Garnet_Solution.cy_Garnet_Garnet_Solution_name())
except AttributeError:
    pass
try:
    print(Garnet_Solution.cy_Garnet_Garnet_Solution_calib_identifier())
    print(Garnet_Solution.cy_Garnet_Garnet_Solution_calib_name())
except AttributeError:
    pass

Test intrinsic element conversion routine ...

In [ ]:
try:
    e = np.zeros(106)
    sum = np.sum(n)
    for index in range(0,c):
        end = Garnet_Solution.cy_Garnet_Garnet_Solution_endmember_elements(index)
        for i in range(0,106):
            e[i] += end[i]*n[index]/sum
    nConv = Garnet_Solution.cy_Garnet_Garnet_Solution_conv_elm_to_moles(e)
    for i in range(0,c):
        print ('X[{0:d}] input {1:13.6e}, calc {2:13.6e}, diff {3:13.6e}'.format(
        i, n[i]/sum, nConv[i], nConv[i]-n[i]/sum))
    if not Garnet_Solution.cy_Garnet_Garnet_Solution_test_moles(nConv):
        print ('Output of intrinsic composition calculation fails tests for permissible values.')
except AttributeError:
    pass
try:
    e = np.zeros(106)
    sum = np.sum(n)
    for index in range(0,c):
        end = Garnet_Solution.cy_Garnet_Garnet_Solution_calib_endmember_elements(index)
        for i in range(0,106):
            e[i] += end[i]*n[index]/sum
    nConv = Garnet_Solution.cy_Garnet_Garnet_Solution_calib_conv_elm_to_moles(e)
    for i in range(0,c):
        print ('X[{0:d}] input {1:13.6e}, calc {2:13.6e}, diff {3:13.6e}'.format(
        i, n[i]/sum, nConv[i], nConv[i]-n[i]/sum))
    if not Garnet_Solution.cy_Garnet_Garnet_Solution_calib_test_moles(nConv):
        print ('Output of intrinsic composition calculation fails tests for permissible values.')
except AttributeError:
    pass

In [ ]:
try:
    print (Garnet_Solution.cy_Garnet_Garnet_Solution_calib_conv_moles_to_tot_moles(n))
    print (Garnet_Solution.cy_Garnet_Garnet_Solution_calib_conv_moles_to_mole_frac(n))
    e = Garnet_Solution.cy_Garnet_Garnet_Solution_calib_conv_moles_to_elm(n)
    print (e)
    print (Garnet_Solution.cy_Garnet_Garnet_Solution_calib_conv_elm_to_moles(e))
    print (Garnet_Solution.cy_Garnet_Garnet_Solution_calib_conv_elm_to_tot_moles(e))
    print (Garnet_Solution.cy_Garnet_Garnet_Solution_calib_conv_elm_to_tot_grams(e))
except AttributeError:
    pass
try:
    print (Garnet_Solution.cy_Garnet_Garnet_Solution_conv_moles_to_tot_moles(n))
    print (Garnet_Solution.cy_Garnet_Garnet_Solution_conv_moles_to_mole_frac(n))
    e = Garnet_Solution.cy_Garnet_Garnet_Solution_conv_moles_to_elm(n)
    print (e)
    print (Garnet_Solution.cy_Garnet_Garnet_Solution_conv_elm_to_moles(e))
    print (Garnet_Solution.cy_Garnet_Garnet_Solution_conv_elm_to_tot_moles(e))
    print (Garnet_Solution.cy_Garnet_Garnet_Solution_conv_elm_to_tot_grams(e))
except AttributeError:
    pass

### Execute the standard thermodynamic property retrieval functions:

In [ ]:
fmt = "{0:<10.10s} {1:13.6e} {2:<10.10s}"
try:
    print(fmt.format('G', Garnet_Solution.cy_Garnet_Garnet_Solution_g(t,p,n), 'J'))
    print(fmt.format('dGdT', Garnet_Solution.cy_Garnet_Garnet_Solution_dgdt(t,p,n), 'J/K'))
    print(fmt.format('dGdP', Garnet_Solution.cy_Garnet_Garnet_Solution_dgdp(t,p,n), 'J/bar'))
    print(fmt.format('d2GdT2', Garnet_Solution.cy_Garnet_Garnet_Solution_d2gdt2(t,p,n), 'J/K^2'))
    print(fmt.format('d2GdTdP', Garnet_Solution.cy_Garnet_Garnet_Solution_d2gdtdp(t,p,n), 'J/K-bar'))
    print(fmt.format('d2GdP2', Garnet_Solution.cy_Garnet_Garnet_Solution_d2gdp2(t,p,n), 'J/bar^2'))
    print(fmt.format('d3GdT3', Garnet_Solution.cy_Garnet_Garnet_Solution_d3gdt3(t,p,n), 'J/K^3'))
    print(fmt.format('d3GdT2dP', Garnet_Solution.cy_Garnet_Garnet_Solution_d3gdt2dp(t,p,n), 'J/K^2-bar'))
    print(fmt.format('d3GdTdP2', Garnet_Solution.cy_Garnet_Garnet_Solution_d3gdtdp2(t,p,n), 'J/K-bar^2'))
    print(fmt.format('d3GdP3', Garnet_Solution.cy_Garnet_Garnet_Solution_d3gdp3(t,p,n), 'J/bar^3'))
    print(fmt.format('S', Garnet_Solution.cy_Garnet_Garnet_Solution_s(t,p,n), 'J/K'))
    print(fmt.format('V', Garnet_Solution.cy_Garnet_Garnet_Solution_v(t,p,n), 'J/bar'))
    print(fmt.format('Cv', Garnet_Solution.cy_Garnet_Garnet_Solution_cv(t,p,n), 'J/K'))
    print(fmt.format('Cp', Garnet_Solution.cy_Garnet_Garnet_Solution_cp(t,p,n), 'J/K'))
    print(fmt.format('dCpdT', Garnet_Solution.cy_Garnet_Garnet_Solution_dcpdt(t,p,n), 'J/K^2'))
    print(fmt.format('alpha', Garnet_Solution.cy_Garnet_Garnet_Solution_alpha(t,p,n), '1/K'))
    print(fmt.format('beta', Garnet_Solution.cy_Garnet_Garnet_Solution_beta(t,p,n), '1/bar'))
    print(fmt.format('K', Garnet_Solution.cy_Garnet_Garnet_Solution_K(t,p,n), 'bar'))
    print(fmt.format('Kp', Garnet_Solution.cy_Garnet_Garnet_Solution_Kp(t,p,n), ''))
except AttributeError:
    pass
try:
    print(fmt.format('G', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_g(t,p,n), 'J'))
    print(fmt.format('dGdT', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_dgdt(t,p,n), 'J/K'))
    print(fmt.format('dGdP', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_dgdp(t,p,n), 'J/bar'))
    print(fmt.format('d2GdT2', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d2gdt2(t,p,n), 'J/K^2'))
    print(fmt.format('d2GdTdP', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d2gdtdp(t,p,n), 'J/K-bar'))
    print(fmt.format('d2GdP2', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d2gdp2(t,p,n), 'J/bar^2'))
    print(fmt.format('d3GdT3', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d3gdt3(t,p,n), 'J/K^3'))
    print(fmt.format('d3GdT2dP', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d3gdt2dp(t,p,n), 'J/K^2-bar'))
    print(fmt.format('d3GdTdP2', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d3gdtdp2(t,p,n), 'J/K-bar^2'))
    print(fmt.format('d3GdP3', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d3gdp3(t,p,n), 'J/bar^3'))
    print(fmt.format('S', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_s(t,p,n), 'J/K'))
    print(fmt.format('V', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_v(t,p,n), 'J/bar'))
    print(fmt.format('Cv', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_cv(t,p,n), 'J/K'))
    print(fmt.format('Cp', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_cp(t,p,n), 'J/K'))
    print(fmt.format('dCpdT', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_dcpdt(t,p,n), 'J/K^2'))
    print(fmt.format('alpha', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_alpha(t,p,n), '1/K'))
    print(fmt.format('beta', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_beta(t,p,n), '1/bar'))
    print(fmt.format('K', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_K(t,p,n), 'bar'))
    print(fmt.format('Kp', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_Kp(t,p,n), ''))
except AttributeError:
    pass

### Execute functions that access endmember properties:

In [ ]:
fmt = "{0:<10.10s} {1:13.6e} {2:<15.15s}"
try:
    print ("number of components", Garnet_Solution.cy_Garnet_Garnet_Solution_endmember_number())
    for index in range(0, c):
        print ("{0:<20.20s}".format(Garnet_Solution.cy_Garnet_Garnet_Solution_endmember_name(index)), end=' ')
        print ("{0:<20.20s}".format(Garnet_Solution.cy_Garnet_Garnet_Solution_endmember_formula(index)))
        print ("mw: {0:10.2f}".format(Garnet_Solution.cy_Garnet_Garnet_Solution_endmember_mw(index)))
        print (fmt.format('mu0', Garnet_Solution.cy_Garnet_Garnet_Solution_endmember_mu0(index,t,p), 'J/mol'))
        print (fmt.format('dmu0dT', Garnet_Solution.cy_Garnet_Garnet_Solution_endmember_dmu0dT(index,t,p), 'J/K-mol'))
        print (fmt.format('dmu0dP', Garnet_Solution.cy_Garnet_Garnet_Solution_endmember_dmu0dP(index,t,p), 'J/bar-mol'))
        print (fmt.format('d2mu0dT2', Garnet_Solution.cy_Garnet_Garnet_Solution_endmember_d2mu0dT2(index,t,p), 'J/K^2-mol'))
        print (fmt.format('d2mu0dTdP', Garnet_Solution.cy_Garnet_Garnet_Solution_endmember_d2mu0dTdP(index,t,p), 'J/K-bar-mol'))
        print (fmt.format('d2mu0dP2', Garnet_Solution.cy_Garnet_Garnet_Solution_endmember_d2mu0dP2(index,t,p), 'J/bar^2-mol'))
        print (fmt.format('d3mu0dT3', Garnet_Solution.cy_Garnet_Garnet_Solution_endmember_d3mu0dT3(index,t,p), 'J/K^3-mol'))
        print (fmt.format('d3mu0dT2dP', Garnet_Solution.cy_Garnet_Garnet_Solution_endmember_d3mu0dT2dP(index,t,p), 'J/K^2-bar-mol'))
        print (fmt.format('d3mu0dTdP2', Garnet_Solution.cy_Garnet_Garnet_Solution_endmember_d3mu0dTdP2(index,t,p), 'J/K-bar^2-mol'))
        print (fmt.format('d3mu0dP3', Garnet_Solution.cy_Garnet_Garnet_Solution_endmember_d3mu0dP3(index,t,p), 'J/bar^3-mol'))
        print ("Element array:")
        print (Garnet_Solution.cy_Garnet_Garnet_Solution_endmember_elements(index))
        print ()
except AttributeError:
    pass
try:
    print ("number of components", Garnet_Solution.cy_Garnet_Garnet_Solution_calib_endmember_number())
    for index in range(0, c):
        print ("{0:<20.20s}".format(Garnet_Solution.cy_Garnet_Garnet_Solution_calib_endmember_name(index)), end=' ')
        print ("{0:<20.20s}".format(Garnet_Solution.cy_Garnet_Garnet_Solution_calib_endmember_formula(index)), end=' ')
        print ("mw: {0:10.2f}".format(Garnet_Solution.cy_Garnet_Garnet_Solution_calib_endmember_mw(index)))
        print (fmt.format('mu0', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_endmember_mu0(index,t,p), 'J/mol'))
        print (fmt.format('dmu0dT', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_endmember_dmu0dT(index,t,p), 'J/K-mol'))
        print (fmt.format('dmu0dP', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_endmember_dmu0dP(index,t,p), 'J/bar-mol'))
        print (fmt.format('d2mu0dT2', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_endmember_d2mu0dT2(index,t,p), 'J/K^2-mol'))
        print (fmt.format('d2mu0dTdP', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_endmember_d2mu0dTdP(index,t,p), 'J/K-bar-mol'))
        print (fmt.format('d2mu0dP2', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_endmember_d2mu0dP2(index,t,p), 'J/bar^2-mol'))
        print (fmt.format('d3mu0dT3', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_endmember_d3mu0dT3(index,t,p), 'J/K^3-mol'))
        print (fmt.format('d3mu0dT2dP', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_endmember_d3mu0dT2dP(index,t,p), 'J/K^2-bar-mol'))
        print (fmt.format('d3mu0dTdP2', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_endmember_d3mu0dTdP2(index,t,p), 'J/K-bar^2-mol'))
        print (fmt.format('d3mu0dP3', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_endmember_d3mu0dP3(index,t,p), 'J/bar^3-mol'))
        print ("Element array:")
        print (Garnet_Solution.cy_Garnet_Garnet_Solution_calib_endmember_elements(index))
        print ()
except AttributeError:
    pass

### Execute functions that access species properties:

In [ ]:
fmt = "{0:<10.10s} {1:13.6e} {2:<15.15s}"
try:
    print ("number of species", Garnet_Solution.cy_Garnet_Garnet_Solution_species_number())
    for index in range(0, c):
        print ("{0:<20.20s}".format(Garnet_Solution.cy_Garnet_Garnet_Solution_species_name(index)), end=' ')
        print ("{0:<20.20s}".format(Garnet_Solution.cy_Garnet_Garnet_Solution_species_formula(index)))
        print ("mw: {0:10.2f}".format(Garnet_Solution.cy_Garnet_Garnet_Solution_species_mw(index)))
        print ("Element array:")
        print (Garnet_Solution.cy_Garnet_Garnet_Solution_species_elements(index))
        print ()
except AttributeError:
    pass
try:
    print ("number of species", Garnet_Solution.cy_Garnet_Garnet_Solution_calib_species_number())
    for index in range(0, c):
        print ("{0:<20.20s}".format(Garnet_Solution.cy_Garnet_Garnet_Solution_calib_species_name(index)), end=' ')
        print ("{0:<20.20s}".format(Garnet_Solution.cy_Garnet_Garnet_Solution_calib_species_formula(index)), end=' ')
        print ("mw: {0:10.2f}".format(Garnet_Solution.cy_Garnet_Garnet_Solution_calib_species_mw(index)))
        print ("Element array:")
        print (Garnet_Solution.cy_Garnet_Garnet_Solution_calib_species_elements(index))
        print ()
except AttributeError:
    pass

### Execute functions for molar derivatives
#### First derivative vectors:

In [ ]:
def printResult(name, result, units):
    print ("{0:<10.10s}".format(name), end=' ')
    [print ("{0:13.6e}".format(x), end=' ') for x in result]
    print ("{0:<10.10s}".format(units))
def printLabels(n):
    print ("{0:<18.18s}".format(''), end=' ')
    [print ("[{0:3d}]{1:<8.8s}".format(idx, ''), end=' ') for idx in range(len(n))]
    print ()
printLabels(n)
try:
    printResult('dGdn', Garnet_Solution.cy_Garnet_Garnet_Solution_dgdn(t,p,n), 'J/m')
    printResult('d2GdndT', Garnet_Solution.cy_Garnet_Garnet_Solution_d2gdndt(t,p,n), 'J/K-m')
    printResult('d2GdndP', Garnet_Solution.cy_Garnet_Garnet_Solution_d2gdndp(t,p,n), 'J/bar-m')
    printResult('d3GdndT2', Garnet_Solution.cy_Garnet_Garnet_Solution_d3gdndt2(t,p,n), 'J/K^2-m')
    printResult('d3GdndTdP', Garnet_Solution.cy_Garnet_Garnet_Solution_d3gdndtdp(t,p,n), 'J/K-bar-m')
    printResult('d3GdndP2', Garnet_Solution.cy_Garnet_Garnet_Solution_d3gdndp2(t,p,n), 'J/bar^2-m')
    printResult('d4GdndT3', Garnet_Solution.cy_Garnet_Garnet_Solution_d4gdndt3(t,p,n), 'J/K^3-m')
    printResult('d4GdndT2dP', Garnet_Solution.cy_Garnet_Garnet_Solution_d4gdndt2dp(t,p,n), 'J/K^2-bar-m')
    printResult('d4GdndTdP2', Garnet_Solution.cy_Garnet_Garnet_Solution_d4gdndtdp2(t,p,n), 'J/K-bar^2-m')
    printResult('d4GdndP3', Garnet_Solution.cy_Garnet_Garnet_Solution_d4gdndp3(t,p,n), 'J/bar^3-m')
except AttributeError:
    pass
try:
    printResult('dGdn', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_dgdn(t,p,n), 'J/m')
    printResult('d2GdndT', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d2gdndt(t,p,n), 'J/K-m')
    printResult('d2GdndP', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d2gdndp(t,p,n), 'J/bar-m')
    printResult('d3GdndT2', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d3gdndt2(t,p,n), 'J/K^2-m')
    printResult('d3GdndTdP', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d3gdndtdp(t,p,n), 'J/K-bar-m')
    printResult('d3GdndP2', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d3gdndp2(t,p,n), 'J/bar^2-m')
    printResult('d4GdndT3', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d4gdndt3(t,p,n), 'J/K^3-m')
    printResult('d4GdndT2dP', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d4gdndt2dp(t,p,n), 'J/K^2-bar-m')
    printResult('d4GdndTdP2', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d4gdndtdp2(t,p,n), 'J/K-bar^2-m')
    printResult('d4GdndP3', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d4gdndp3(t,p,n), 'J/bar^3-m')
except AttributeError:
    pass    

#### The Hessian matrix (molar second derivative matrix) is stored as a compact linear array
A function is provided to map matrix indices to compact storage 1-D array indices

In [ ]:
for i in range(1,c+1):
    print ("[ ", end=' ')
    for j in range (1,c+1):
        print ((i,j), end=' ')
    print (']     [', end=' ')
    for j in range (1,c+1):
        print (model.symmetric_index_from_2d_array(elm=(i,j)), end=' ')
    print (']')

In [ ]:
def printResult(name, result, units):
    print ("{0:<10.10s}".format(name), end=' ')
    [print ("{0:13.6e}".format(x), end=' ') for x in result]
    print ("{0:<10.10s}".format(units))
def printLabels(n):
    print ("{0:<18.18s}".format(''), end=' ')
    maxIdx = int(len(n)*(len(n)-1)/2 + len(n))
    [print ("[{0:3d}]{1:<8.8s}".format(idx, ''), end=' ') for idx in range(maxIdx)]
    print ()
printLabels(n)
try:
    printResult('d2Gdn2', Garnet_Solution.cy_Garnet_Garnet_Solution_d2gdn2(t,p,n), 'J/m^2')
    printResult('d3Gdn2dT', Garnet_Solution.cy_Garnet_Garnet_Solution_d3gdn2dt(t,p,n), 'J/K-m^2')
    printResult('d3Gdn2dP', Garnet_Solution.cy_Garnet_Garnet_Solution_d3gdn2dp(t,p,n), 'J/bar-m^2')
    printResult('d4Gdn2dT2', Garnet_Solution.cy_Garnet_Garnet_Solution_d4gdn2dt2(t,p,n), 'J/K^2-m^2')
    printResult('d4Gdn2dTdP', Garnet_Solution.cy_Garnet_Garnet_Solution_d4gdn2dtdp(t,p,n), 'J/K-bar-m^2')
    printResult('d4Gdn2dP2', Garnet_Solution.cy_Garnet_Garnet_Solution_d4gdn2dp2(t,p,n), 'J/bar^2-m^2')
    printResult('d5Gdn2dT3', Garnet_Solution.cy_Garnet_Garnet_Solution_d5gdn2dt3(t,p,n), 'J/K^3-m^2')
    printResult('d5Gdn2dT2dP', Garnet_Solution.cy_Garnet_Garnet_Solution_d5gdn2dt2dp(t,p,n), 'J/K^2-bar-m^2')
    printResult('d5Gdn2dTdP2', Garnet_Solution.cy_Garnet_Garnet_Solution_d5gdn2dtdp2(t,p,n), 'J/K-bar^2-m^2')
    printResult('d5Gdn2dP3', Garnet_Solution.cy_Garnet_Garnet_Solution_d5gdn2dp3(t,p,n), 'J/bar^3-m^2')
except AttributeError:
    pass
try:
    printResult('d2Gdn2', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d2gdn2(t,p,n), 'J/m^2')
    printResult('d3Gdn2dT', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d3gdn2dt(t,p,n), 'J/K-m^2')
    printResult('d3Gdn2dP', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d3gdn2dp(t,p,n), 'J/bar-m^2')
    printResult('d4Gdn2dT2', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d4gdn2dt2(t,p,n), 'J/K^2-m^2')
    printResult('d4Gdn2dTdP', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d4gdn2dtdp(t,p,n), 'J/K-bar-m^2')
    printResult('d4Gdn2dP2', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d4gdn2dp2(t,p,n), 'J/bar^2-m^2')
    printResult('d5Gdn2dT3', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d5gdn2dt3(t,p,n), 'J/K^3-m^2')
    printResult('d5Gdn2dT2dP', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d5gdn2dt2dp(t,p,n), 'J/K^2-bar-m^2')
    printResult('d5Gdn2dTdP2', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d5gdn2dtdp2(t,p,n), 'J/K-bar^2-m^2')
    printResult('d5Gdn2dP3', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d5gdn2dp3(t,p,n), 'J/bar^3-m^2')
except AttributeError:
    pass

#### The 3-D Tensor (molar third derivative tensor) is stored as a compact linear array
A function is provided to map matrix indices to compact storage 1-D array indices:  
If $n_c$ represents the number of components in the solution, and  
if $n_d$ represents the dimensionality of molar derivative (in this case 3), then  
the number of numerically ordered permutations of $n_c$ molar derivatives taken $n_d$ at a time is:

In [ ]:
n_c,n_d = sym.symbols('n_c n_d')
q = sym.factorial(n_c+n_d-1)/sym.factorial(n_d)/sym.factorial(n_c-1)
q

Substituting $n_d$ equal to 3 and simplifying gives:

In [ ]:
q = sym.simplify(q.subs(n_d,3))
q

and, for the number of components in this solution, there will be the following number of unique terms in the third derivative tensor:

In [ ]:
q.subs(n_c,c)

A function is provided to map matrix indices to compact storage 1-D array indices

In [ ]:
for i in range(1,c+1):
    for j in range (1,c+1):
        print ("[", end=' ')
        for k in range (1,c+1):
            print ("{0:1d}{1:1d}{2:1d}".format(i,j,k), end=' ')
        print ('] ', end=' ')
    print ('  ->  ', end=' ')
    for j in range (1,c+1):
        print ("[", end=' ')
        for k in range (1,c+1):
            print (model.symmetric_index_from_3d_array(elm=(i,j,k)), end=' ')
        print ('] ', end=' ')
    print ('')

In [ ]:
def printResult(name, result, units):
    print ("{0:<10.10s}".format(name), end=' ')
    [print ("{0:10.3e}".format(x), end=' ') for x in result]
    print ("{0:<14.14s}".format(units))
def printLabels(n):
    print ("{0:<15.15s}".format(''), end=' ')
    maxIdx = int(len(n)*(len(n)+1)*(len(n)+2)/6)
    [print ("[{0:3d}]{1:<5.5s}".format(idx, ''), end=' ') for idx in range(maxIdx)]
    print ()
printLabels(n)
try:
    printResult('d3Gdn3', Garnet_Solution.cy_Garnet_Garnet_Solution_d3gdn3(t,p,n), 'J/m^3')
    printResult('d4Gdn3dT', Garnet_Solution.cy_Garnet_Garnet_Solution_d4gdn3dt(t,p,n), 'J/K-m^3')
    printResult('d4Gdn3dP', Garnet_Solution.cy_Garnet_Garnet_Solution_d4gdn3dp(t,p,n), 'J/bar-m^3')
    printResult('d5Gdn3dT2', Garnet_Solution.cy_Garnet_Garnet_Solution_d5gdn3dt2(t,p,n), 'J/K^2-m^3')
    printResult('d5Gdn3dTdP', Garnet_Solution.cy_Garnet_Garnet_Solution_d5gdn3dtdp(t,p,n), 'J/K-bar-m^3')
    printResult('d5Gdn3dP2', Garnet_Solution.cy_Garnet_Garnet_Solution_d5gdn3dp2(t,p,n), 'J/bar^2-m^3')
    printResult('d6Gdn3dT3', Garnet_Solution.cy_Garnet_Garnet_Solution_d6gdn3dt3(t,p,n), 'J/K^3-m^3')
    printResult('d6Gdn3dT2dP', Garnet_Solution.cy_Garnet_Garnet_Solution_d6gdn3dt2dp(t,p,n), 'J/K^2-bar-m^3')
    printResult('d6Gdn3dTdP2', Garnet_Solution.cy_Garnet_Garnet_Solution_d6gdn3dtdp2(t,p,n), 'J/K-bar^2-m^3')
    printResult('d6Gdn3dP3', Garnet_Solution.cy_Garnet_Garnet_Solution_d6gdn3dp3(t,p,n), 'J/bar^3-m^3')
except AttributeError:
    pass
try:
    printResult('d3Gdn3', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d3gdn3(t,p,n), 'J/m^3')
    printResult('d4Gdn3dT', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d4gdn3dt(t,p,n), 'J/K-m^3')
    printResult('d4Gdn3dP', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d4gdn3dp(t,p,n), 'J/bar-m^3')
    printResult('d5Gdn3dT2', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d5gdn3dt2(t,p,n), 'J/K^2-m^3')
    printResult('d5Gdn3dTdP', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d5gdn3dtdp(t,p,n), 'J/K-bar-m^3')
    printResult('d5Gdn3dP2', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d5gdn3dp2(t,p,n), 'J/bar^2-m^3')
    printResult('d6Gdn3dT3', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d6gdn3dt3(t,p,n), 'J/K^3-m^3')
    printResult('d6Gdn3dT2dP', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d6gdn3dt2dp(t,p,n), 'J/K^2-bar-m^3')
    printResult('d6Gdn3dTdP2', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d6gdn3dtdp2(t,p,n), 'J/K-bar^2-m^3')
    printResult('d6Gdn3dP3', Garnet_Solution.cy_Garnet_Garnet_Solution_calib_d6gdn3dp3(t,p,n), 'J/bar^3-m^3')
except AttributeError:
    pass

## Test and time the generated functions for Garnet

Time the code

In [ ]:
try:
    %timeit(Garnet_Solution.cy_Garnet_Garnet_Solution_g(t, p, n))
except AttributeError:
    pass
try:
    %timeit(Garnet_Solution.cy_Garnet_Garnet_Solution_calib_g(t, p, n))
except AttributeError:
    pass

Time the Rubicon wrapped Objective-C code

In [ ]:
from thermoengine import model as stdmodel
modelDB = stdmodel.Database()
GarnetHC = modelDB.get_phase('Grt')

In [ ]:
%timeit(GarnetHC.gibbs_energy(t,p,mol=n))

## Methods available only in the "Calib" versions of generated code
### Execute the parameter value/metadata functions.  
These functions are only defined for the "calibration" model code implementation:

In [ ]:
nparam = 0

In [ ]:
Garnet_Solution.cy_Garnet_Garnet_Solution_get_param_number()
try:
    nparam = Garnet_Solution.cy_Garnet_Garnet_Solution_get_param_number()
    names = Garnet_Solution.cy_Garnet_Garnet_Solution_get_param_names()
    units = Garnet_Solution.cy_Garnet_Garnet_Solution_get_param_units()
    values = Garnet_Solution.cy_Garnet_Garnet_Solution_get_param_values()
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,nparam):
        print(fmt.format(names[i], values[i], Garnet_Solution.cy_Garnet_Garnet_Solution_get_param_value(i), units[i]))
except AttributeError:
    pass

### Functions that allow modification of the array of parameter values

In [ ]:
try:
    values[1] = 100.0
    Garnet_Solution.cy_Garnet_Garnet_Solution_set_param_values(values)
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,nparam):
        print(fmt.format(names[i], values[i], Garnet_Solution.cy_Garnet_Garnet_Solution_get_param_value(i), units[i]))
except (AttributeError, NameError):
    pass

### Functions that allow modification of a particular parameter value

In [ ]:
try:
    Garnet_Solution.cy_Garnet_Garnet_Solution_set_param_value(1, 1.0)
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,nparam):
        print(fmt.format(names[i], values[i], Garnet_Solution.cy_Garnet_Garnet_Solution_get_param_value(i), units[i]))
except AttributeError:
    pass

### Functions that evaluate parameter derivatives ...

In [ ]:
try:
    fmt = "    {0:<10.10s} {1:13.6e}"
    for i in range(0, np):
        print ('Derivative with respect to parameter: ', names[i], ' of')
        print (fmt.format('G', Garnet_Solution.cy_Garnet_Garnet_Solution_dparam_g(t, p, n, i)))
        print (fmt.format('dGdT', Garnet_Solution.cy_Garnet_Garnet_Solution_dparam_dgdt(t, p, n, i)))
        print (fmt.format('dGdP', Garnet_Solution.cy_Garnet_Garnet_Solution_dparam_dgdp(t, p, n, i)))
        print (fmt.format('d2GdT2', Garnet_Solution.cy_Garnet_Garnet_Solution_dparam_d2gdt2(t, p, n, i)))
        print (fmt.format('d2GdTdP', Garnet_Solution.cy_Garnet_Garnet_Solution_dparam_d2gdtdp(t, p, n, i)))
        print (fmt.format('d2GdP2', Garnet_Solution.cy_Garnet_Garnet_Solution_dparam_d2gdp2(t, p, n, i)))
        print (fmt.format('d3GdT3', Garnet_Solution.cy_Garnet_Garnet_Solution_dparam_d3gdt3(t, p, n, i)))
        print (fmt.format('d3GdT2dP', Garnet_Solution.cy_Garnet_Garnet_Solution_dparam_d3gdt2dp(t, p, n, i)))
        print (fmt.format('d3GdTdP2', Garnet_Solution.cy_Garnet_Garnet_Solution_dparam_d3gdtdp2(t, p, n, i)))
        print (fmt.format('d3GdP3', Garnet_Solution.cy_Garnet_Garnet_Solution_dparam_d3gdp3(t, p, n, i)))
except (AttributeError, TypeError):
    pass

### Parameter derivatives of the chemical potential

In [ ]:
def printResult(name, result, units):
    print ("dmu[*]/d {0:<10.10s}".format(name), end=' ')
    [print ("{0:13.6e}".format(x), end=' ') for x in result]
    print ("{0:<12.12s}".format(units))
def printLabels(n):
    print ("         {0:<18.18s}".format(''), end=' ')
    [print ("[{0:3d}]{1:<8.8s}".format(idx, ''), end=' ') for idx in range(len(n))]
    print ()
printLabels(n)
try:
    for i in range(0, nparam):
        result = Garnet_Solution.cy_Garnet_Garnet_Solution_dparam_dgdn(t,p,n, i)
        printResult(names[i], result, 'J/m^2/p-unit')
except AttributeError:
    pass    